In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
pip install cv2

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Feature Extraction (Deep Features)
import cv2
import numpy as np
import pandas as pd
import re
import math
from skimage import feature
from skimage.feature import hog
from imutils import paths
from google.colab.patches import cv2_imshow
import os

from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

# Load Inception_v3 pretrained on ImageNet dataset
W=224
H=224
#model = InceptionV3(include_top=False, weights='imagenet', pooling='avg', input_tensor=Input(shape=(W,H,3)))
#model = ResNet50(include_top=False, weights='imagenet', pooling='avg', input_tensor=Input(shape=(W,H,3)))
model = Xception(include_top=False, weights='imagenet', pooling='avg')

# List of paths 
import os
file_list=[]
file_list.append(os.listdir(r"/home/jorge/OneDrive/GitHub/University_Projects/Machine Learning/Class11/fake"))
file_list.append(os.listdir(r"/home/jorge/OneDrive/GitHub/University_Projects/Machine Learning/Class11/real"))

# general path
path='/home/jorge/OneDrive/GitHub/University_Projects/Machine Learning/Class11/'

# list of classes
class_names=['fake', 'real'] 

X_deep = []
y = []

# Gera lista com as imagens
for classes_files, classe in zip (file_list, range(2)):
    for i in range(len(classes_files)):
      name= str(path) + str(class_names[classe]) + str('/') + str(classes_files[i]) 
#      print(name)
      y.append(classe)
      
# Carrega a imagem, pré-processa e inclui em uma lista 
      imagem = cv2.imread(name)
      img = cv2.resize(imagem,(W,H))
      xd = image.img_to_array(img)
      xd = np.expand_dims(xd, axis=0)
      xd = preprocess_input(xd)
      X_deep.append(xd)

# Extrai características usando o modelo profundo pré-treinado (deep learning)
X_deep = np.asarray(X_deep)
X_deep = X_deep.reshape(X_deep.shape[0], W, H, 3)
X =  model.predict(X_deep)      

# Salva as características extraídas em um csv (um vetor de valores para cada imagem)
df = pd.DataFrame(X)
df.to_csv('X_im.csv', header=False, index=False)

# Salva y que contém a classe de cada imagem
df_class = pd.DataFrame(y)
df_class.to_csv('y_im.csv', header=False, index=False)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
# Exemplo de como carregar os arquivos gerados
import pandas as pd
import numpy as np

# Labels
y = pd.read_csv('y_im.csv', header=None)
y=y.to_numpy()
y=np.ravel(y)
print(y.shape)

# deep features
X = pd.read_csv('X_im.csv', header=None)
X=X.to_numpy()
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split

# Definição dos parâmetros a serem avaliados
parameters=[{'hidden_layer_sizes':[1000, (1000,500)], 
             'activation':['relu', 'logistic'],}]

# Separar uma parte dos dados para validação
#X, X_val, y, y_val=train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

# definição da técnica a ser utilizada
mlp=MLPClassifier(random_state=46, max_iter=500, hidden_layer_sizes=(1000, 500))

#gs=GridSearchCV(mlp, parameters, cv=5)
#gs.fit(X_val, y_val)
#mlp=gs.best_estimator_
#print("Melhores parâmetros:", gs.best_params_)

#from tabulate import tabulate
#df=gs.cv_results_
#print(tabulate(df, headers='keys', tablefmt='psql'))

result=model_selection.cross_val_score(mlp, X, y, cv=5)
print("Accuracy: %.5f" % result.mean())
print("std: %.5f" % result.std())
y_pred=model_selection.cross_val_predict(mlp, X, y, cv=5)
ma=confusion_matrix(y, y_pred)
print("Confusion Matrix:")
print(ma)